#### __Các bước thực hiện bài tập 2.3__ 
- $\text{Khởi tạo SparkSession}$
- $\text{Đọc dữ liệu và gán shapeid}$
- $\text{Thu thập dữ liệu về Driver và chuyển sang Polygon}$
- $\text{Tìm các cặp hình có giao nhau thực sự}$
- $\text{Tạo DataFrame kết quả và sắp xếp}$
- $\text{Lưu kết quả về file output.csv}$

#### __Các thư viện sử dụng__

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from shapely.geometry import Polygon

##### $\text{Khởi tạo SparkSession}$ 

In [2]:
spark = SparkSession.builder \
    .appName("Shape Overlap Detection") \
    .getOrCreate()

##### $\text{Đọc dữ liệu và gán shapeid}$

In [3]:
# Đọc dữ liệu từ file parquet
input_path = "shapes.parquet"
raw_df = spark.read.parquet(input_path)

# Gán shape_id nếu bị NULL
df = raw_df.withColumn("shape_id", monotonically_increasing_id())

##### $\text{Thu thập dữ liệu về Driver và chuyển sang Polygon}$

In [4]:
# Thu thập dữ liệu về driver
shape_data = df.select("shape_id", "vertices").collect()

# Chuyển đổi vertices thành Polygon
polygon_list = []
for row in shape_data:
    shape_id = row["shape_id"]
    vertices = [(x, y) for x, y in row["vertices"]]
    polygon = Polygon(vertices)
    polygon_list.append((shape_id, polygon))

##### $\text{Tìm các cặp hình có giao nhau thực sự}$

In [5]:
# Tìm các cặp polygon giao nhau
overlap_pairs = []
n_polygons = len(polygon_list)

for i in range(n_polygons):
    id1, poly1 = polygon_list[i]

    for j in range(i + 1, n_polygons):
        id2, poly2 = polygon_list[j]
        intersection = poly1.intersection(poly2)
        if  intersection.area > 0:
            overlap_pairs.append((int(id1), int(id2)))

##### $\text{Tạo DataFrame kết quả và sắp xếp}$

In [6]:
# Tạo Spark DataFrame từ kết quả
columns = ["shape_1", "shape_2"]
result_df = spark.createDataFrame(overlap_pairs, columns)

# Sắp xếp kết quả
result_df = result_df.orderBy("shape_1", "shape_2")
# In ra kết quả
result_df.show(truncate=False)

+-------+-------+
|shape_1|shape_2|
+-------+-------+
|0      |41     |
|0      |143    |
|0      |198    |
|0      |430    |
|0      |473    |
|0      |490    |
|0      |496    |
|0      |605    |
|0      |637    |
|0      |729    |
|0      |818    |
|1      |118    |
|1      |160    |
|1      |186    |
|1      |220    |
|1      |247    |
|1      |256    |
|1      |377    |
|1      |444    |
|1      |809    |
+-------+-------+
only showing top 20 rows



In [7]:
#in số dòng của dataframe
print("Số dòng của dataframe:", result_df.count())

Số dòng của dataframe: 6562


##### $\text{Lưu kết quả về file output.csv}$

In [8]:
# Sao chép dữ liệu từ Spark DataFrame
rows = result_df.collect()

columns = ["shape_1", "shape_2"]
output_path = "./output.csv"

with open(output_path, "w") as file:
    # Ghi dòng tiêu đề
    file.write(",".join(columns) + "\n")

    # Ghi từng dòng dữ liệu
    for row in rows:
        values = [str(row[col]) for col in columns]
        file.write(",".join(values) + "\n")
